
# **Laboratorium 002: Skuteczne promptowanie modeli LLM w Ollama**

## **Cel zajęć**

- Wyćwiczenie praktycznych technik **prompt engineeringu** na lokalnych modelach uruchamianych przez **Ollama**.  
- Zastosowanie **struktur i ról** (system/developer, user, assistant) w praktyce – z naciskiem na *kompozycję promptów*, a nie pojedyncze pytania.  
- Przeprowadzenie **ewaluacji promptów** (PromptFoo) i analizy zachowania modeli.  
- Zrozumienie zasad **bezpieczeństwa, prompt injection, anonimizacji** i dobrych praktyk projektowych.

---

## **1. Przygotowanie środowiska i prerekwizyty**

- Zainstalowana **Ollama** i dwa modele w konwencji *polski – niepolski* (`llama3.1:8b`, `gemma2:2b`, `SpeakLeash/bielik-*`).  

Test działania modelu:

```bash
echo "Napisz rymowankę o najlepszym wykładowcy na WI." | ollama run gemma2:2b
```
---

- Możemy też przenieść się z pracą do notebooka, wtedy nasz warsztat będzie wyglądał tak:

In [ ]:
import ollama

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': 'Napisz rymowankę o najlepszym wykładowcy na WI'}]
)
print(response['message']['content'])

## **2. Struktura i techniki promptowania**

### 2.1 Zero-shot

**Zero‑shot prompting** to sposób formułowania zapytania do dużego modelu językowego, w którym nie podajemy mu ani jednego przykładu wykonanego zadania. Model dostaje wyłącznie instrukcję („zrób X”) i całą resztę dedukuje sam na podstawie wiedzy nabytej w treningu.

Zadania, do których używa się **zero-shot prompting**:
1. Szybkie prototypowanie – gdy chcemy sprawdzić, czy model może poradzić sobie z danym zadaniem (czy jego architektura i materiał treningowy na to pozwalają).

2. Zadania ogólne o prostym formacie wyjściowym (tłumaczenia, parafrazy, proste klasyfikacje).

3. Systemy low-code/no-code – użytkownik pisze naturalnym językiem, a model odpowiada bez konfiguracji.

4. Ekstrakcja informacji _ad hoc_ (imię, mail, daty z tekstu itp.) tam, gdzie nie opłaca się trenować lub fine-tune'ować modelu.


Kiedy **zero-shot** jest najbardziej efektywne?
- brak lub szczątkowe dane oznaczone – nie mamy przykładowych par wejście→wyjście.
- zadanie jest podobne do scenariuszy widzianych w treningu (np. „podsumuj”, „przetłumacz”, „odpowiedz TAK/NIE”).
- liczy się czas wdrożenia – chcemy odpowiedzi „tu i teraz”, bez skrupulatnego modyfikowania promptu.

Kiedy nie wystarcza?
- złożone transformacje wymagające ścisłej struktury lub niestandardowych reguł.
- zadania domenowe z mało popularnym żargonem (medycyna, prawo) – wtedy lepszy jest few-shot lub RAG.

```bash
echo "Wypisz 5 słów kluczowych z notatki o spotkaniu Koła Naukowego BIT." | ollama run gemma2:2b
```

🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

- Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

- Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

- Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

- Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

 - Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

- Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

- Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

- Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

- Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl

Zadanie 2.1.1 Generowanie streszczenia

In [ ]:
text = """
🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl
"""

prompt = f"""Return a JSON with one key "summary" containing
a 30‑word English summary of the following text:

{text}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': prompt}]
)
print(response['message']['content'])

Zadanie 2.1.2 Otrzymywanie odpowiedzi od modelu

Poprzednie laboratoria były poświęcone głównie zapytaniom typu zero-shot, którymi weryfikuje się działanie modeli.

In [ ]:
# Pytanie weryfikujące wiedzę i pewność modelu
response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': 'jaka jest pogoda w warszawie'}]
)
print(response['message']['content'])

Zadanie 2.1.3

Używając odpowiednich technik promptowania, możemy wymusić odpowiedź na modelu. Posłużmy się więc powyższym przykładem i rozbudujmy odpowiedź modelu, wykorzystując schemat JSON.

In [ ]:
from pydantic import BaseModel

class CityWeather(BaseModel):
    city: str
    temp_c: float
    condition: str | None

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role':'user','content':'Weather in Warsaw today.'}],
    format=CityWeather.model_json_schema(),   # ← schema trafia do Ollamy
    options={'temperature':0}
)

weather = CityWeather.model_validate_json(response['message']['content'])
print(weather)

Podobne pytanie w PowerShellu:
```PowerShell
@'
<ROLE>Jesteś asystentem danych. Zwracasz tylko JSON.</ROLE>
<GOAL>Wyodrębnij dane liczbowe i jednostki.</GOAL>
<RULES>- Jeśli brak danych, zwróć null.</RULES>
<INPUT>Jan kupił 3 jabłka po 2 zł.</INPUT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

> **Pytania pomocnicze**  
> - Jakiej odpowiedzi oczekiwałeś/aś?  
> - Czy model zachował format JSON?  
> - W jaki inny sposób można wymusić przestrzeganie formatu?

Dodatkowe wskazówki:

1. Nie otaczaj JSON-u blokiem json – przy format="json" to zbędne, a istnieje ryzyko, że model uzna ``` za legalny token i walidacja się przerwie.

2. Daj modelowi „awaryjną” ścieżkę – np. If you can’t comply, output {}. Zmniejsza szansę na halucynacje.

3. Limit długości – przy dużych strukturach dodaj max_tokens, żeby model nie wypadł poza kontekst i nie zamknął nawiasu.

4. Testuj → parsuj → próbuj od nowa – w kodzie API zawsze parsuj JSON i w razie JSONDecodeError ponawiaj z tym samym promptem; przy format='json' błędy i tak zdarzają się rzadko.

5. Użytkownik-czat – gdy korzystasz interaktywnie (bez API), nie masz formatu; wtedy najpewniejszy miks to system-instrukcja + <json>…</json> + temp=0.

---
### 2.2 Few-shot prompting

Dodając **przykłady** w promptach (na różnych poziomach), wskazujemy modelowi pożądany wzorzec.

Zadanie 2.2.1  
Przygotuj co najmniej dwa przykłady (pytanie → odpowiedź) i poproś model o wygenerowanie odpowiedzi dla nowego pytania w tym samym stylu.


In [ ]:
examples = [
    {"role": "user", "content": "Translate to emoji: I love programming"},
    {"role": "assistant", "content": "💻❤️"},
    {"role": "user", "content": "Translate to emoji: Fire and ice"},
    {"role": "assistant", "content": "🔥❄️"},
    {"role": "user", "content": "Translate to emoji: Peace and coffee. Trophy and fast car. Running and winning."}
]

response = ollama.chat(model='gemma2:2b', messages=examples)
print(response['message']['content'])


Zadanie 2.2.2 Ekstrakcja danych

In [ ]:
examples = [
    {"role": "user", "content": 'Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."'},
    {"role": "assistant", "content": '{"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}'},
    {"role": "user", "content": 'Wejście: "Jutro o 9:30 na Teamsach."'},
    {"role": "assistant", "content": '{"data":"jutro","godzina":"09:30","miejsce":"Teams"}'},
    {"role": "user", "content": 'Wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."'}
]

response = ollama.chat(model='gemma2:2b', messages=examples)
print(response['message']['content'])

Podobna struktura zapytań w PowerShellu:
```bash

@'
Przykład 1:
Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."
Wyjście: {"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}

Przykład 2:
Wejście: "Jutro o 9:30 na Teamsach."
Wyjście: {"data":"jutro","godzina":"09:30","miejsce":"Teams"}

Nowe wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."
'@ | ollama run llama3.1:8b
```


Zadanie 2.2.3 Ekstrakcja danych do formatu JSON

In [ ]:
import json

schema = """Return valid JSON: { "name": <string>, "title": <string>, "company": <string> }"""

shots = [
    # przykład 1
    {"role": "user",      "content": schema + "\n— Sarah Connors-Newman — Director of Operations at Skynet Industries"},
    {"role": "assistant", "content": '{"name":"Sarah Connors-Newman","title":"Director of Operations","company":"Skynet Industries"}'},
    # przykład 2
    {"role": "user",      "content": schema + "\n— Dr. Hiro Tanaka, Lead Scientist • QuantumX"},
    {"role": "assistant", "content": '{"name":"Hiro Tanaka","title":"Lead Scientist","company":"QuantumX"}'},
    # przykład 3
    {"role": "user",      "content": schema + "\n— Prof. dr hab. Janek Dzbanek Przewodnik Wszystkich AGH"},
    {"role": "assistant", "content": '{"name":"Janek Dzbanek","title":"Przewodnik Wszystkich","company":"AGH"}'},
]

text = "• Prof. dr hab. Janek Tanaka wysadził AGH w powietrze"

response = ollama.chat(
    model="gemma2:2b",
    messages=shots + [
        {"role": "user", "content": schema + "\n" + text}
    ],
    format="json",              
    options={"temperature": 0}
)

print(json.loads(response["message"]["content"]))

---
## 3. Role i wiadomości systemowe

W Ollama (jak w OpenAI) możemy dodać komunikat **`system`**, który definiuje rolę lub ograniczenia modelu.

Zadanie 3.1 Definiowanie roli

Zdefiniuj rolę **nauczyciela języka polskiego** i poproś o poprawienie błędów w zdaniu ucznia.


In [ ]:
messages = [
    {"role": "system", "content": "You are a strict but encouraging Polish language teacher."},
    {"role": "user", "content": "Popraw proszę to zdanie: 'Wczoraj byłem w kinie i oglądali film.'"}
]

response = ollama.chat(model='gemma2:2b', messages=messages)
print(response['message']['content'])

## 4. Chain‑of‑Thought (CoT)

**Chain‑of‑Thought** zachęca model do wypisywania kroków rozumowania.

Zadanie 4.1 Wyjaśnianie kroków rozumowania

Poproś model, aby rozwiązał zagadkę logiczną i podał kroki rozumowania, a na końcu hasło w linii `ANSWER: ...`.

In [ ]:
puzzle = """Jestem liczbą dwucyfrową. Moja suma cyfr to 9, a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"""

prompt = f"""Let's solve this step‑by‑step.

{puzzle}

Format:
STEP 1: ...
STEP 2: ...
ANSWER: <number>
"""

response = ollama.chat(model='gemma2:2b', messages=[{'role':'user','content':prompt}])
print(response['message']['content'])


Zadanie 4.2 Wyjaśnianie kroków rozumowania z przypisanymi rolami systemowymi

Poproś model, by rozwiązał zagadkę logiczną. Rozpisz zadania dla poszczególnych elementów systemu.



In [ ]:
puzzle = (
    "Jestem liczbą dwucyfrową. Moja suma cyfr to 9, "
    "a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"
)

SYSTEM = (
    "You are a careful math tutor. "
    "Explain EVERY algebraic step explicitly, without skipping or merging steps. "
    "Never combine two operations into one line; label them as separate STEP n."
)

USER = f"""
Solve the puzzle **step-by-step**.

{puzzle}

Output format (exactly):
STEP 1: …
STEP 2: …
…
ANSWER: <number>
Do NOT skip, abbreviate, or summarise any step.
"""

resp = ollama.chat(
    model="gemma2:2b",
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user",   "content": USER}, 
    ]
)

print(resp["message"]["content"])

## 5. Bezpieczeństwo

Nie istnieje obecnie skuteczny sposób całkowitego zabezpieczenia modeli przed zmianą ich zachowania przez użytkownika końcowego — potwierdzają to badania nad Constitutional Classifiers (https://www.anthropic.com/research/constitutional-classifiers). Dlatego systemy z LLM należy projektować tak, by nawet udany atak nie powodował poważnych skutków.

Podstawowe zasady bezpieczeństwa:

- Model nie może samodzielnie wykonywać decyzji biznesowych ani nieodwracalnych akcji.

- Każde działanie powinno być zatwierdzane przez człowieka (Human in the Loop).

- Dostępy i uprawnienia należy kontrolować programistycznie, nie poprzez prompt.

- System powinien mieć jasne komunikaty, regulaminy i ograniczenia prawne.

Typowe ryzyka:

- Czatbot z bazą wiedzy może tworzyć błędne lub niestosowne treści.

- Czatbot z dostępem do narzędzi może przypadkowo lub celowo usuwać, modyfikować lub wysyłać dane.
  → Rozwiązanie: ogranicz uprawnienia, prowadź historię zmian i wymagaj potwierdzeń akcji.

Ataki Prompt Injection mogą wynikać nie tylko ze złych intencji, lecz także z błędnej konstrukcji systemu lub nieporozumienia modelu.
Dlatego wszystkie generowane treści należy weryfikować i zatwierdzać przez człowieka.

> Celem nie jest całkowita izolacja modeli, lecz świadome ograniczanie ryzyka i kontrola efektów ich działania.

# **Walidacja promptów**

`PromptFoo` to narzędzie open-source do testowania, porównywania i walidacji promptów dla modeli językowych (np. ChatGPT, Ollama, Claude, Gemini, itp.).
Pozwala automatycznie oceniać, które prompty generują najlepsze, najbardziej spójne lub najdokładniejsze odpowiedzi.

Dzięki niemu możesz:

- uruchamiać testy wielu promptów jednocześnie,

- porównywać modele lokalne (Ollama) i zdalne (OpenAI API),

- definiować własne asercje (reguły walidacji), np. „odpowiedź nie jest pusta”, „zawiera słowo kluczowe”, „ma poprawny JSON”.

Instalacja PromptFoo
1. Wymagania

Node.js w wersji ≥18

```bash
node -v
```

Jeśli nie masz Node.js — pobierz go z https://nodejs.org/

2. Instalacja PromptFoo

Zainstaluj za pomocą npm:

```bash
npm install -g promptfoo
```

3. Sprawdzenie instalacji

Po zakończeniu instalacji wpisz:

```bash
promptfoo --version
```

Jeśli pojawi się numer wersji — wszystko działa

4. Utworzenie przykładowego projektu

W dowolnym folderze uruchom:

```bash
promptfoo init
```

To polecenie utworzy plik konfiguracyjny promptfooconfig.yaml i przykładowe testy.

5. Uruchomienie testów promptów

Aby wykonać testy i zobaczyć wyniki:

```bash
promptfoo eval
promptfoo view
```

promptfoo view otworzy interaktywny panel w przeglądarce.

> Oficjalna strona: https://www.promptfoo.dev

Przykładowy metaprompt do PromptFoo

```bash
<ROLE>
Jesteś ekspertem od Prompt Engineeringu. Twoim zadaniem jest pomóc użytkownikowi stworzyć skuteczny prompt
dla modelu językowego (LLM), który zapewni dokładne, zwięzłe i powtarzalne odpowiedzi.
</ROLE>

<GOAL>
Na podstawie opisu zadania od użytkownika:
1. Zidentyfikuj cel promptu i format odpowiedzi.
2. Zaproponuj strukturę promptu (z sekcjami <ROLE>, <RULES>, <INPUT>, <OUTPUT>).
3. Dodaj 1–2 przykłady (few-shot), które pokażą wzorzec zachowania.
4. Zaproponuj sposób ewaluacji jego skuteczności (np. z użyciem PromptFoo).
</GOAL>

<INPUT>
{{opis_zadania_użytkownika}}
</INPUT>

<OUTPUT>
Zwróć gotowy prompt i krótkie uzasadnienie.
</OUTPUT>
```

------

## **7. Zadania sprawdzające do sprawozdania**

Do każdego zadania ułóż swoje przykłady i napisz swoje prompty.

Zadanie 7.1 NER (Named Entity Recognition)

Rozpoznawanie nazw własnych (osoba, organizacja, lokalizacja, data).
Zwróć wynik w formacie JSON.

przykład:

```python
SYSTEM = "You are an information extractor."

prompt = "Return valid JSON listing every PERSON, LOCATION and ORGANIZATION that appears."

text = """7 lutego 1919, dekretem Tymczasowego Naczelnika Państwa Józefa Piłsudskiego, 
      utworzona została Pocztowa Kasa Oszczędności. Jej pierwszym dyrektorem został mianowany 28 grudnia 1919 Hubert Linde. 
      Po nim prezesami PKO byli Emil Schmidt i Henryk Gruber. Z czasem utworzono centralę banku w Warszawie 
      z siedzibą przy ul. Świętokrzyskiej 31/33 oraz pierwsze oddziały lokalne: w Krakowie, Lwowie, Łodzi, Poznaniu i Katowicach. 
      Pierwszym celem PKO stało się wprowadzenie do obiegu polskiego złotego zamiast marki polskiej (jako pochodnej 
      marki niemieckiej). Od 1920 bank posiadał osobowość prawną, jako instytucja państwowa. Pracownicy Kasy byli zrzeszeni 
      w Zrzeszeniu Pracowników Pocztowej Kasy Oszczędności, które miało swoje koła przy większych Oddziałach, np. w Warszawie, 
      w Łodzi."""

output_format = """
{
  "PERSON": ["<imię nazwisko>"],
  "LOCATION": ["<miejsce>"],
  "ORG": ["<organizacja>"],
}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"{prompt}\n\nText:\n{text}\n\nOutput format:\n{output_format}"}
    ]
)
print(response['message']['content'])
```
---

Zadanie 7.2 Analiza sentymentu

Określenie tonu wypowiedzi (pozytywny, neutralny, negatywny).
Dodaj przykład sarkazmu i porównaj interpretacje modeli.

przykład: 

```python
SYSTEM = "Jesteś precyzyjnym analizatorem wydźwięku tekstu. Zwracaj wyłącznie poprawny JSON według podanej specyfikacji."

prompt = """Określ wydźwięk (pozytywny/negatywny/neutralny) poniższej recenzji.
Zwróć JSON z dwoma kluczami: "sentiment" i "evidence" (zacytuj decydujący fragment, ogranicz się do trzech najbardziej emocjonalnych wyrazów).

Recenzja:
Gdy pierwszy raz uniosłam do nosa butelkę Szamponu „Lśniąca Natura”, ogarnęła mnie fala wspomnień z wakacji nad Bałtykiem – zapach morskiej bryzy splecionej z nutami słodkiej pomarańczy i soczystych malin. Już samo otwarcie opakowania było jak zdjęcie wiecznego klosza z codzienności: w jednej sekundzie łazienka zamieniła się w rozświetloną, letnią plażę, a ja – w beztroską dziewczynę z wiatrem we włosach.

Gęsta, perłowa formuła wypływa z butelki niczym płynne światło. Kiedy rozprowadzam ją na wilgotnych pasmach, mam wrażenie, że każdy kosmyk wita ją z zachwytem: pianę miękką jak pianka z latte, która lekko skrzypi między palcami i otula skórę głowy kojącym chłodem. To nie jest zwykłe mycie włosów – to rytuał, w którym czuję się dopieszczona od cebulek aż po same końce.

Już podczas spłukiwania słyszę charakterystyczny, czysty „skrzyp” zdrowych włosów. Strumień wody odbija światło, a moje pasma – lśniące i lekkie – tańczą w nim jak jedwabne wstążki. Nie mogę się oprzeć, by nie zanurzyć dłoni w tej tafli – gładkość rozczarowuje mnie tylko w jednym: że nie da się jej zapisać na stałe w pamięci dotyku.

Po wysuszeniu czuję się, jakbym stąpała po czerwonym dywanie: pukle sypkie, sprężyste, unoszące się przy każdym ruchu głowy. Aromat, który pozostaje, przypomina delikatny perfum – dyskretny, ale wystarczająco wyrazisty, by ktoś obok zapytał z zaciekawieniem: „Czym pachniesz?” Wtedy uśmiecham się szeroko, bo wiem, że ten sekret kryje się w niewielkiej, zielonej butelce stojącej na półce.

Szampon „Lśniąca Natura” to dla mnie nie tylko kosmetyk; to codzienny list miłosny do moich włosów – powiew odwagi i czułości zarazem. Jeśli Twoje pasma pragną rozgwieżdżonego blasku, a Twoje zmysły tęsknią za chwilą szczerej przyjemności, pozwól temu szamponowi szepnąć im historię o tym, jak piękno rodzi się z zachwytu.

"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ]
)
print(response['message']['content'])
```

---

Zadanie 7.3 Ekstrakcja relacji

Wykrywanie relacji PRACUJE_W i MIESZKA_W.

przykład:

```python
SYSTEM = "Jesteś precyzyjnym analizatorem tekstu. Znajdź wszystkie relacje gdzie osoba pracuje dla organizacji. Zwróć tablicę JSON z obiektami {\"person\":\"<imię>\",\"company\":\"<organizacja>\"}."

shots = [
    {
        "role": "user", 
        "content": "Tekst: \"Jan Kowalski pracuje w Microsoft jako programista.\""
    },
    {
        "role": "assistant",
        "content": "[{\"person\":\"Jan Kowalski\",\"company\":\"Microsoft\"}]"
    },
    {
        "role": "user",
        "content": "Tekst: \"Anna Kowalska teraz pracuje na AGH, ale kiedyś karierę robiła w Google.\""
    }
]

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        *shots
    ]
)
print(response['message']['content'])
```

---

Zadanie 7.4 Klasyfikacja tematyczna

Przypisz tekst do jednej z kategorii: nauka, sport, polityka, technologia.
Przetestuj trzy warianty: zero-shot, few-shot, sekcyjny.

```python
SYSTEM = (
    "Jesteś klasyfikatorem tematów. Przypisz tekst do JEDNEJ kategorii "
    "z zestawu: nauka, sport, polityka, technologia. "
    "Zwróć wyłącznie JSON: {\"kategoria\":\"...\"}."
)

text = 'Nowy mikroskop pozwala obserwować pojedyncze atomy.'

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Tekst: \"{text}\""}
    ]
)

print(response['message']['content'])
'@ | ollama run gemma2:2b
```

---

Zadanie 7.5 Streszczenie i parafraza

Generuj dwa streszczenia (krótkie i długie) oraz parafrazę krótkiego.

```bash
@'
<ROLE>Tworzysz streszczenia i parafrazy. Odpowiadasz w JSON.</ROLE>
<TEXT>
Sztuczna inteligencja wspiera naukowców w analizie danych medycznych,
pomagając szybciej diagnozować choroby i tworzyć spersonalizowane terapie.
</TEXT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

---

Zadanie 7.6: QA z kontekstem i filtrowaniem odpowiedzi

**Opis zadania:**

Przygotuj prompt, który pozwoli modelowi LLM (w Ollama) odpowiadać na pytania zadane przez użytkownika, na podstawie dostarczonego kontekstu (tekstu źródłowego). Oceniane zadania:

1. Zaprojektowanie struktury promptu z rolami (system/user) i sekcjami:
   - **Kontekst**: fragment tekstu, z którego model może czerpać informacje.
   - **Pytanie użytkownika**.
   - **Zasady**: na przykład — „jeśli pytanie nie może być odpowiedziane z kontekstu, odpowiedz: ‘Brak wystarczających informacji’”, „podaj źródło w nawiasach [] jeśli to możliwe”.
2. Przeprowadzenie kilku eksperymentów z wariantami promptu:
   - wariant *bez zasad*,
   - wariant *ze ścisłymi zasadami*,
   - wariant z **few-shot** przykładem (kontekst + pytanie + przykładowa poprawna odpowiedź).
3. Dla zadanych par (kontekst + pytanie) porównać odpowiedzi z różnych wariantów promptów, ocenić poprawność, rozbieżności i halucynacje.
4. (Dodatkowe) napisać testy w `PromptFoo`, by automatycznie sprawdzać, czy:
   - odpowiedź nie jest pusta,
   - jeśli pytanie dotyczy czegoś niezawartego w kontekście, to model faktycznie odpowiada „Brak wystarczających informacji” (lub inny ustalony komunikat).

**Przykład formatu (schemat):**

```
<system>
Jesteś asystentem, który odpowiada na pytania na podstawie dostarczonego tekstu. Jeśli pytanie wykracza poza tekst, odpowiedz „Brak wystarczających informacji”.
</system>

<kontekst>
{tutaj wklej fragment tekstu}
</kontekst>

<user>
Pytanie: {tutaj pytanie}
</user>
```

**Materiały do testów:**

Przygotuj:

- pytanie, na które odpowiedź jest w tekście,
- pytanie spoza tekstu.

------

Zadanie 7.7: Porównanie strategii promptów w klasyfikacji wieloklasowej + analiza

**Opis zadania:**

Celem jest zbadanie i porównanie efektywności różnych strategii promptów (zero-shot, few-shot, oddzielone sekcjami) w zadaniu klasyfikacji wieloklasowej tekstu, a następnie analiza wyników (precision / recall / błędy) i wyciągnięcie wniosków.

1. Przygotuj zbiór testowy (np. 20 krótkich zdań) i zbiór klas (np. 4–5 tematów: „technologia”, „sport”, „zdrowie”, „polityka”).
2. Zaimplementuj co najmniej trzy warianty promptu:
   - **Zero-shot**: zadanie z instrukcją i klasami, bez przykładów.
   - **Few-shot**: dwa/trzy przykłady (tekst + klasa) w promptcie, a potem nowe wejścia.
   - **Sekcje / separatory**: z oddzieloną sekcją „Zasady”, „Dane wejściowe”, „Wynik” – np. strukturą RULES, INPUT, OUTPUT.
3. Uruchom klasyfikację na wszystkich wariantach promptów (ten sam model i parametry), zbierz odpowiedzi.
4. Zanalizuj wyniki:
   - policz *accuracy*, *konfuzje między klasami* (które klasy najczęściej pomylono),
   - sprawdź, dla których przykładów różne promptowania dały różne odpowiedzi – spróbuj zdiagnozować, dlaczego.
   - opcjonalnie: zmodyfikuj prompt (np. zmiana kolejności instrukcji, dodanie więcej przykładów) i sprawdź, czy następuje poprawa.
5. (Bonus) Zintegruj testy w `PromptFoo`, by automatycznie weryfikować trafność klasyfikacji (np. asercje *equals(expected_class)* dla części przypadków).

------

Zadanie 7.8: Mini-projekt: Asystent planowania wyjazdu

Stwórz **asystenta turystycznego**, który:

1. Przyjmuje nazwę miasta w Polsce,
2. Zwraca plan jednodniowego zwiedzania (poranne, popołudniowe, wieczorne atrakcje),
3. Uwzględnia ograniczenie budżetu przekazane przez użytkownika,
4. Zwraca wynik w formacie Markdown z nagłówkami `### Rano`, `### Popołudnie`, `### Wieczór`.

## **Dalsze materiały**

- Gandalf (https://gandalf.lakera.ai/baseline)
- Ollama Docs (https://docs.ollama.com/)
- Prompt Engineering Guide (https://www.promptingguide.ai/)
- PromptFoo / LangFuse (https://www.promptfoo.dev/) / (https://langfuse.com/)
- Anthropic Research (https://www.anthropic.com/research)
- Publikacje o CoT, Self-Critique i Prompt Injection

> **Refleksja:** Która technika promptowania dała najbardziej stabilne rezultaty i jak można zoptymalizować proces w kolejnych projektach?